## تدريب نموذج CBoW

هذا الدفتر هو جزء من [منهج الذكاء الاصطناعي للمبتدئين](http://aka.ms/ai-beginners)

في هذا المثال، سنستعرض تدريب نموذج لغة CBoW للحصول على مساحة تضمين Word2Vec الخاصة بنا. سنستخدم مجموعة بيانات AG News كمصدر للنصوص.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

أولاً، دعنا نحمل مجموعة البيانات الخاصة بنا ونعرّف المُجزئ والمفردات. سنضبط `vocab_size` على 5000 لتقليل العمليات الحسابية قليلاً.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## نموذج CBoW

يتعلم نموذج CBoW التنبؤ بكلمة بناءً على الكلمات المجاورة التي يبلغ عددها $2N$. على سبيل المثال، عندما يكون $N=1$، سنحصل على الأزواج التالية من الجملة *I like to train networks*: (like,I)، (I, like)، (to, like)، (like,to)، (train,to)، (to, train)، (networks, train)، (train,networks). هنا، الكلمة الأولى هي الكلمة المجاورة المستخدمة كمدخل، والكلمة الثانية هي التي نتنبأ بها.

لبناء شبكة للتنبؤ بالكلمة التالية، سنحتاج إلى تقديم الكلمة المجاورة كمدخل، والحصول على رقم الكلمة كمخرج. هيكل شبكة CBoW هو كالتالي:

* يتم تمرير الكلمة المدخلة عبر طبقة التضمين. هذه الطبقة نفسها ستكون تضمين Word2Vec الخاص بنا، لذلك سنقوم بتعريفها بشكل منفصل كمتغير `embedder`. سنستخدم حجم تضمين = 30 في هذا المثال، على الرغم من أنك قد ترغب في تجربة أبعاد أعلى (Word2Vec الحقيقي يحتوي على 300).
* يتم تمرير متجه التضمين بعد ذلك إلى طبقة خطية تتنبأ بالكلمة الناتجة. لذلك تحتوي على عدد من الخلايا العصبية يساوي `vocab_size`.

بالنسبة للمخرجات، إذا استخدمنا `CrossEntropyLoss` كدالة خسارة، فسيتعين علينا أيضًا تقديم أرقام الكلمات فقط كنتائج متوقعة، دون استخدام الترميز الواحد الساخن.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## إعداد بيانات التدريب

الآن دعونا نبرمج الدالة الرئيسية التي ستقوم بحساب أزواج الكلمات باستخدام نموذج CBoW من النص. هذه الدالة ستتيح لنا تحديد حجم النافذة، وستعيد مجموعة من الأزواج - الكلمة المدخلة والكلمة الناتجة. لاحظ أن هذه الدالة يمكن استخدامها على الكلمات، وكذلك على المتجهات/التنسورات - مما سيمكننا من ترميز النص قبل تمريره إلى دالة `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


لنُحضّر مجموعة بيانات التدريب. سنقوم بمراجعة جميع الأخبار، وندعو `to_cbow` للحصول على قائمة أزواج الكلمات، ونضيف تلك الأزواج إلى `X` و `Y`. من أجل توفير الوقت، سنأخذ فقط أول 10 آلاف خبر - يمكنك بسهولة إزالة هذا القيد إذا كان لديك المزيد من الوقت للانتظار وترغب في الحصول على تمثيلات أفضل :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

سنقوم أيضًا بتحويل تلك البيانات إلى مجموعة بيانات واحدة، وإنشاء محمل بيانات:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

سنقوم أيضًا بتحويل تلك البيانات إلى مجموعة بيانات واحدة، وإنشاء محمل بيانات:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

الآن دعونا نبدأ التدريب الفعلي. سنستخدم مُحسّن `SGD` بمعدل تعلم مرتفع جدًا. يمكنك أيضًا تجربة استخدام مُحسّنات أخرى، مثل `Adam`. سنقوم بالتدريب لمدة 10 عصور كبداية - ويمكنك إعادة تشغيل هذه الخلية إذا كنت تريد خسارة أقل.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## تجربة Word2Vec

لاستخدام Word2Vec، دعونا نستخرج المتجهات التي تمثل جميع الكلمات في مفرداتنا:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

دعونا نرى، على سبيل المثال، كيف يتم ترميز كلمة **باريس** إلى متجه:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


من المثير للاهتمام استخدام Word2Vec للبحث عن المرادفات. الوظيفة التالية ستعيد `n` أقرب الكلمات إلى الإدخال المعطى. للعثور عليها، نقوم بحساب معيار $|w_i - v|$، حيث $v$ هو المتجه المقابل للكلمة المدخلة، و $w_i$ هو الترميز للكلمة رقم $i$ في المفردات. ثم نقوم بترتيب المصفوفة وإرجاع الفهارس المقابلة باستخدام `argsort`، ونأخذ أول `n` عناصر من القائمة، التي ترمز إلى مواقع الكلمات الأقرب في المفردات.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## خلاصة

باستخدام تقنيات ذكية مثل CBoW، يمكننا تدريب نموذج Word2Vec. يمكنك أيضًا تجربة تدريب نموذج skip-gram الذي يتم تدريبه على توقع الكلمة المجاورة بناءً على الكلمة المركزية، وملاحظة مدى أدائه.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
